In [229]:
import os
from dotenv import load_dotenv
load_dotenv()

GROK_API_KEY=os.getenv("GROK_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(api_key=GROK_API_KEY, model_name="Gemma2-9b-It")
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import ToolExecutor

In [ ]:
from langchain.tools import tool

@tool
def calculator_tool(input: str) -> str:
    """Solve basic math expressions. Input should be a math string like '2+2'."""
    try:
        result = eval(input)
        return str(result)
    except Exception as e:
        return f"Calculator Error: {str(e)}"


from langchain.tools import tool

@tool
def webscraper_tool(url: str) -> str:
    """Simulate web scraping for a URL."""
    return f"Simulated content from {url}"


from langchain.tools import tool

@tool
def mock_tool(input: str) -> str:
    """Mock tool for testing purposes. Just echoes the input."""
    return f"Mock response for: {input}"
@tool
def transfer_to_tools_Agent():
    """Ask tool_Agent for help."""
    return
@tool
def transfer_to_plan_Agent():
    """Ask plan_Agent for help."""
    return


from langchain_community.tools.tavily_search import TavilySearchResults
@tool
def search(query: str) -> str:
    """Perform the web search on the user query."""
    tavily = TavilySearchResults()
    result = tavily.invoke(query)
    return result


In [231]:

from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool as AgentTool


In [232]:

llm_new = llm.bind_tools([transfer_to_tools_Agent, transfer_to_plan_Agent, calculator_tool, webscraper_tool, mock_tool,search])

In [233]:
def plan_agent_fn(state):
    """
    Plans tasks iteratively based on input. It adds, deletes, or modifies tasks.
    """
    query = state['input']
    history = state.get("history", [])

    prompt = f"""
You are a planning agent. Your job is to break down the user input into clear steps (tasks).
Make sure to: 
- Split tasks logically
- Add or modify based on previous errors
- Output a SINGLE task to do next (one at a time)

User Input:
{query}

History: {history}
"""
    response = llm([HumanMessage(content=prompt)])
    content = response.content.strip()

    return {
        "task": content,
        "input": query,
        "result": state.get("result", ""),
        "history": history
    }


In [ ]:

from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool 

from tools import calculator_tool
from tools import webscraper_tool
from tools import mock_tool
from tools import search

tools = [
    Tool.from_function(
        func=calculator_tool,
        name="Calculator",
        description="Useful for performing basic math operations"
    ),
    Tool.from_function(
        func=webscraper_tool,
        name="WebScraper",
        description="Useful for answering questions by scraping the web"
    ),
    Tool.from_function(
        func=mock_tool,
        name="MockTool",
        description="A mock tool for testing purposes"
    ),
    Tool.from_function(
        func=search,
        name="Search",
        description="Perform a web search on the user query"
    )
]


agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def tool_agent_fn(state: dict) -> dict:
    """
    Executes the task using the best matching tool.
    """
    task = state.get("task")
    if not task:
        return {"result": "No task provided."}

    # try:
    #     result = llm_new.invoke(task)
    #     return {"result": result}

    try:
        result = agent.run(task)
        return {"result": result}
    except Exception as e:
        return {"result": f"ToolAgent Error: {str(e)}"}


In [235]:
class FeedbackAgent:
    def refine_task(self, task, result):
        if "error" in result:
            
            return task + " (refined)"
        return task

In [236]:
from typing_extensions import Literal
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph,START,END
from langgraph.types import Command
from dotenv import load_dotenv
from IPython.display import Image, display
# from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
# from langchain_experimental.utilities import PythonREPL

In [ ]:
def decide_to_generate(state):
    result = state.get("result", "").lower()
    if "error" in result:
        return "plan"  
    else:
        return "tools"  


    # task = state.get("task", "").lower()
    
    # # You can put any logic here
    # if "calculator" in task or "search" or "calculate" in task:
    #     return "tools" 
    # else:
    #     return "End"


In [238]:
def llm_checker_fn(state):
    tool_output = state.get("result", "")
    print("🔍 Received tool result:", tool_output)

    if not tool_output:
        return {"result": "⚠️ No tool output found."}

    llm_response = llm.invoke(f"Review this tool output and rewrite it like an article: {tool_output}")
    
    return {
        "result": llm_response.content
    }


In [ ]:

from langgraph.graph import StateGraph
from typing import Annotated, Sequence, TypedDict


# state = {"input": str, "task": str, "result": str, "history": list}
class AgentState(TypedDict):
    input: str
    task: str
    result: str
    history: list

# LangGraph structure
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("PlanAgent", plan_agent_fn)
workflow.add_node("ToolAgent", tool_agent_fn)
# workflow.add_node("LLMChecker", llm_checker_fn)

# workflow.add_node("FeedbackAgent", feedback_agent_fn)


workflow.set_entry_point("PlanAgent")
workflow.add_edge("PlanAgent", "ToolAgent")
workflow.add_conditional_edges(
    "PlanAgent",
    decide_to_generate,
    {
        "plan": "PlanAgent",
        "tools": "ToolAgent",
    },
)
workflow.add_edge("ToolAgent", END)
# workflow.add_edge("ToolAgent", "LLMChecker")


# workflow.add_edge("LLMChecker", END)

# workflow.add_edge("FeedbackAgent", "PlanAgent")  # loop until success
# workflow.add_edge("ToolAgent", "PlanAgent")  # loop until success

# Finalize
graph = workflow.compile(
    # checkpointer=memory,
    # This is new!
    # interrupt_before=["FeedbackAgent"],  # interrupt before feedback agent
    # Note: can also interrupt __after__ tools, if desired.
    # interrupt_after=["tools"]
)


In [ ]:
import pprint
input_state = {
    "input": "who is virat kohli?",
    "task": "",  # Add task field
    "result": "",  # Add result field
    "history": []  # Keep history
}


for output in graph.stream(input_state):
    for key, value in output.items():
            pprint.pprint(f"Output from node '{key}':")
            pprint.pprint("---")
            pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

"Output from node 'PlanAgent':"
'---'
{ 'history': [],
  'input': 'who is virat kohli?',
  'result': '',
  'task': 'TASK: Search for information about Virat Kohli on the internet.'}
'\n---\n'


> Entering new AgentExecutor chain...
Thought: I need to use the Search tool to find information about Virat Kohli. 
Action: Search
Action Input: Virat Kohli
Observation: [{'url': 'https://en.wikipedia.org/wiki/Virat_Kohli', 'content': "Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ⓘ; born 5 November 1988) is an Indian international cricketer who plays Test and ODI cricket for the Indian national team. A former captain in all formats of the game, Kohli retired from the T20I format following India's win at the 2024 T20 World Cup. He's a right-handed batsman and an occasional unorthodox right arm medium pace bowler. Kohli holds the highest IPL run-scorer record, ranks third in T20I, third in ODI, and stands the [...] Kohli was a member of the Indian team that won the 2008 Under-19 Cricket Wo